In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
path2SP = "/Users/carlostoruno/OneDrive - World Justice Project/EU Subnational/EU-S Data"
survey_groups = ["AA", "AB", "BA", "BB"]

eugpp = pd.read_stata(
    f"{path2SP}/eu-gpp/1. Data/3. Merge/EU_GPP_2024.dta", 
    convert_categoricals = False
).set_index("country_year_id")

kmeans_results =[
    pd.read_csv(f"../../../data/kmeans_results/{g}_kmeans_results.csv")
    for g in survey_groups
]
kmeans_data = pd.concat(kmeans_results).set_index("country_year_id")

outline = (
    pd.read_excel("../../../metadata/theoretical_outline.xlsx")
)
outline_vars = list(set(outline.target_var))

master_data = (
    kmeans_data
    .merge(
        eugpp.loc[:, outline_vars],
        right_index=True,
        left_index=True,
        how="left"
    )
)
master_data[outline_vars] = master_data[outline_vars].replace([98,99], np.nan)